# Bayesian Structural Time Series: Forecasting and Decomposition Using Facebook's Prophet 

이 노트북은 Facebook Prophet을 사용하여 시계열 분해/예측을 어떻게 수행하는지 보여준다. Prophet은 내부적으로 Baysian Structural Time Series(BSTS) 구조를 가지지만, 단순 인터페이스 제공을 위해 대부분의 BSTS는 숨겨져있다.

BSTS 예를 보기 위해 전기 수요 데이터를 사용한다.

## Data
`tensor-house-data` 저장소의 데이터셋을 참고한다.

---

In [ ]:
from matplotlib import pylab as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd
import numpy as np
from fbprophet import Prophet

demand_dates = np.arange('2014-01-01', '2014-02-26', dtype='datetime64[h]')
demand_loc = mdates.WeekdayLocator(byweekday=mdates.WE)
demand_fmt = mdates.DateFormatter('%a %b %d')

data = pd.read_csv('../../../tensor-house-data/time-series/electricity-demand-victoria.csv', comment='#', header=None).T
data = pd.concat([data, pd.DataFrame(demand_dates)], axis=1)
data.columns = ["demand", "temperature", "date"]

# 데이터 분리
num_forecast_steps = 24 * 7 * 2 # two weeks
data_training = data[:-num_forecast_steps]
data_test = data[-num_forecast_steps:]

colors = sns.color_palette()
c1, c2 = colors[0], colors[1]

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(2, 1, 1)
ax.plot(data_training.date,
        data_training.demand, lw=2, label="training data")
ax.set_ylabel("Hourly demand (GW)")

ax = fig.add_subplot(2, 1, 2)

ax.plot(data_training.date,
        data_training.temperature, lw=2, label="training data", c=c2)
ax.set_ylabel("Temperature (deg C)")
ax.set_title("Temperature")
ax.xaxis.set_major_locator(demand_loc)
ax.xaxis.set_major_formatter(demand_fmt)
fig.suptitle("Electricity Demand in Victoria, Australia (2014)", fontsize=15)
fig.autofmt_xdate()


#
# Forecasting and decomposition using Prophet
#
df = data_training.rename(columns={'date':'ds', 'demand':'y'})

# 학습
m = Prophet(yearly_seasonality=False)
m.fit(df)

# 테스트
future = m.make_future_dataframe(periods=num_forecast_steps, freq='H')
forecast = m.predict(future)
fig1 = m.plot(forecast)
fig2 = m.plot_components(forecast)